In [1]:
import pandas as pd
import numpy as np
import pickle

In [ ]:

# List of file paths
file_paths = ['./data/raw/census/numeric/census-bronx.csv', 
              './data/raw/census/numeric/census-kings.csv', 
              './data/raw/census/numeric/census-newyork.csv', 
              './data/raw/census/numeric/census-queens.csv', 
              './data/raw/census/numeric/census-richmond.csv']

# List to store dataframes
dfs = []

# Import CSV files as dataframes
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Join dataframes column-wise
census = pd.concat(dfs, axis=1)



In [3]:
#extract column
labels = np.array(census.iloc[:, 0])

# Remove 'Labels (Grouping)' column from census dataframe
census = census.drop(columns=['Label (Grouping)'])

# Convert all columns to numeric
census = census.apply(pd.to_numeric, errors='coerce')

#add column back
census.insert(0, 'Race', labels)


In [4]:
census = census.drop(index=[1, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 70])

In [5]:
# invert make rows columns and columns rows
census = census.transpose()

In [6]:
# give columns name of first row
census.columns = census.iloc[0]
census = census[1:]

In [7]:
# remove whitepaces from column names
census.columns = census.columns.str.strip()

In [8]:
# Rename race columns to standarized
col_names = {
    'Total:': 'Total',
    'White alone': 'white',
    'Black or African American alone': 'Black',
    'American Indian and Alaska Native alone': 'American Indigenous',
    'Asian alone': 'Asian2',
    'Native Hawaiian and Other Pacific Islander alone': 'Asian1',
    'Some Other Race alone': 'Other',
    'Population of two races:': 'Mixed Race1',
    'Population of three races:': 'Mixed Race2',
    'Population of four races:': 'Mixed Race3',
    'Population of five races:': 'Mixed Race4',
    'Population of six races:': 'Mixed Race5'
}
 
census = census.rename(columns=col_names)

In [9]:
# combine mixed race columns
census['Mixed Race'] = census[['Mixed Race5', 'Mixed Race1', 'Mixed Race2', 'Mixed Race3', 'Mixed Race4']].sum(axis=1)


In [10]:
# combine Asian columns
census['Asian'] = census[['Asian2', 'Asian1']].sum(axis=1)

In [11]:
census.drop(columns=['Asian2', 'Asian1', 'Mixed Race5', 'Mixed Race1', 'Mixed Race2', 'Mixed Race3', 'Mixed Race4'], inplace=True)


In [12]:
census = census.reset_index()
census = census.rename(columns={'index': 'BoroCT2020'})


# Convert to Census tract numbers

In [13]:
# make a new boro column
census['BoroName'] = census['BoroCT2020'].str.extract(r', (.+?) County')

In [14]:
# cut to just the number
census['BoroCT2020'] = census['BoroCT2020'].astype(str)
census['BoroCT2020'] = census['BoroCT2020'].str.extract(r'Census Tract ([\d.]+),')

In [15]:
# remove period if the string has one, add two 00s to the end if it doesnt
census['BoroCT2020'] = census['BoroCT2020'].apply(lambda x: x.replace('.', '') if '.' in x else x + '00')

In [16]:
# add zeros to front of string until reach length of 6 total
census['BoroCT2020'] = census['BoroCT2020'].str.zfill(6)

In [17]:
# Update BoroCT2020 and BoroName to match census geo data
census.loc[census['BoroName'] == 'New York', 'BoroCT2020'] = '1' + census.loc[census['BoroName'] == 'New York', 'BoroCT2020']
census.loc[census['BoroName'] == 'New York', 'BoroName'] = 'Manhattan'

census.loc[census['BoroName'] == 'Richmond', 'BoroCT2020'] = '5' + census.loc[census['BoroName'] == 'Richmond', 'BoroCT2020']
census.loc[census['BoroName'] == 'Richmond', 'BoroName'] = 'Staten Island'

census.loc[census['BoroName'] == 'Kings', 'BoroCT2020'] = '3' + census.loc[census['BoroName'] == 'Kings', 'BoroCT2020']
census.loc[census['BoroName'] == 'Kings', 'BoroName'] = 'Brooklyn'

census.loc[census['BoroName'] == 'Queens', 'BoroCT2020'] = '4' + census.loc[census['BoroName'] == 'Queens', 'BoroCT2020']

census.loc[census['BoroName'] == 'Bronx', 'BoroCT2020'] = '2' + census.loc[census['BoroName'] == 'Bronx', 'BoroCT2020']

# Replace NAs with zeros

In [18]:
# change numeric NaN values to 0
census = census.fillna(0)

In [19]:
# calculate a total values for all rows
if (census['Total'] == 0).any():
    # Sum all numeric rows and replace the 0 with the total
    census['Total'] = census.select_dtypes(include='number').sum(axis=1)

In [ ]:
# write to csv
census.to_csv('./data/processed/numeric/census.csv', index=False)

In [ ]:
# Define the file path
output_filepath = './data/processed/numeric/census.pkl'

# Open the file in write mode
with open(output_filepath, 'wb') as file:
    # Save the processed DataFrame as a pickle object
    pickle.dump(census, file)